# Robustness of a Regression Model

In this notebook, we will evaluate the robustness of a regression model on the "US Crimes" dataset, to perform the analysis. We will use the Linear Regression Gradient Descent Based poisoner to poison the dataset and evaluate the robustness of the model to the poison data. We will use the Mean Squared Error (MSE) as the evaluation metric.

In [1]:
import warnings

import numpy as np
import pandas as pd
from sklearn import linear_model
from holisticai.datasets import load_dataset
from sklearn.metrics import mean_squared_error
from holisticai.robustness.attackers import LinRegGDPoisoner

warnings.filterwarnings("ignore")


## Loading the dataset

We will use the "US Crimes" dataset, which contains crime statistics for each US state. The target variable is the number of crimes per 100,000 people, and the features are the different crime statistics.

We will use the preprocessed version of the dataset, which is available in the `load_dataset` module of the `holisticai` library.

In [2]:
dataset = load_dataset('us_crime', preprocessed=True)
train_test = dataset.train_test_split(test_size=0.25, random_state=42)

train = train_test['train']
test = train_test['test']
train, test

(<holisticai.datasets._dataset.Dataset at 0x79b57c30fd00>,
 <holisticai.datasets._dataset.Dataset at 0x79b57c30fd90>)

## Preprocessing the data

Since the 'fold' column is used for stratified cross-validation, we will remove it from the input features of the dataset.

In [3]:
X_train = train['X'].drop(columns=['fold'])
X_test = test['X'].drop(columns=['fold'])

y_train = train['y']
y_test = test['y']

X_train.head()

,state,population,householdsize,racepctblack,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,agePct65up,...,NumStreet,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LandArea,PopDens,PctUsePubTrans,LemasPctOfficDrugUn
0,25,0.00,0.42,0.04,0.13,0.03,0.25,0.42,0.23,0.30,...,0.0,0.20,0.69,0.60,0.72,0.70,0.03,0.08,0.18,0.0
1,42,0.00,0.50,0.04,0.03,0.01,0.37,0.39,0.26,0.36,...,0.0,0.07,0.91,0.85,0.88,0.86,0.04,0.06,0.02,0.0
2,34,0.00,0.67,0.03,0.40,0.03,0.35,0.33,0.23,0.40,...,0.0,0.31,0.53,0.77,0.64,0.69,0.02,0.16,0.54,0.0
3,25,0.01,0.41,0.01,0.03,0.02,0.29,0.33,0.22,0.60,...,0.0,0.24,0.77,0.78,0.80,0.78,0.01,0.37,0.54,0.0
4,6,0.05,0.51,0.08,0.47,0.47,0.41,0.53,0.34,0.33,...,0.0,0.47,0.52,0.49,0.85,0.78,0.02,0.52,0.16,0.0


## Training the regression model

We will train a linear regression model on the training data to use as a baseline model.

In [4]:
clf = linear_model.LinearRegression()
clf.fit(X_train, y_train)

baseline_error = mean_squared_error(y_test, clf.predict(X_test))

print(f"Baseline error: {baseline_error}")

Baseline error: 0.018695360634785115


## Gradient Descent Based Poisoning

We will now generate poison data using the Linear Regression Gradient Descent Based poisoner. This poisoner uses the training data and the gradient descent algorithm to generate poison data and then, we will use the poison data to evaluate the robustness of the model and how it affects the model's performance. A recommended practice is to use a small fraction (no more than 0.2) of the training data to generate the poison data that will be appended to the training data at the end.

To do that, first, we will create a categorical mask to tell the poisoner which features will be treated as categorical. Then, we will create the poisoner object and use the `generate` method to generate the poison data.

In [5]:
categorical_mask = np.zeros(X_train.shape[1])
categorical_mask[0] = 0

In [6]:
poiser = LinRegGDPoisoner(poison_proportion=0.2, num_inits=1) #  

# Poison the training data
x_poised, y_poised = poiser.generate(X_train, y_train, categorical_mask = categorical_mask, return_only_poisoned=False)

2024-09-17 09:24:16,973 - INFO - Training Error: 0.043305
2024-09-17 09:24:16,976 - INFO - Best initialization error: 0.043305
2024-09-17 09:24:16,977 - INFO - Poison Count: 374.000000
2024-09-17 09:24:17,015 - INFO - Iteration 0:
2024-09-17 09:24:17,018 - INFO - Objective Value: 0.04330518226687288 Change: 0.04330518226687288
2024-09-17 09:24:47,991 - INFO - Iteration 1:
2024-09-17 09:24:47,994 - INFO - Objective Value: 0.04775163750034216 Change: 0.0044464552334692806
2024-09-17 09:24:47,996 - INFO - Y pushed out of bounds: 346/374
2024-09-17 09:25:19,915 - INFO - Iteration 2:
2024-09-17 09:25:19,917 - INFO - Objective Value: 0.04826614832819969 Change: 0.0005145108278575283
2024-09-17 09:25:19,921 - INFO - Y pushed out of bounds: 344/374
2024-09-17 09:25:47,975 - INFO - Iteration 3:
2024-09-17 09:25:47,979 - INFO - Objective Value: 0.0479650035114509 Change: -0.00030114481674878985
2024-09-17 09:25:47,980 - INFO - Y pushed out of bounds: 348/374
2024-09-17 09:25:47,982 - INFO - no p

**Evaluating Model Robustness**

We will evaluate the robustness of the model by training the model on the poisoned data and evaluating its performance on the test data. We will compare the performance of the model on the test data.

In [7]:
clfp = linear_model.LinearRegression()
clfp.fit(x_poised, y_poised)

poised_err = mean_squared_error(y_test, clfp.predict(X_test))

print("Error before poisoning:", baseline_error)
print("Error after poisoning:", poised_err)

Error before poisoning: 0.018695360634785115
Error after poisoning: 0.05898545946820494


As we can see from the results, the model's performance on the test data is significantly worse when trained on the poisoned data. This demonstrates the importance of evaluating the robustness of a model to poison data.